Imports:

In [16]:
import torch
from torch import nn  # All neural network modules
from torch import optim  # For optimizers like SGD, Adam, etc.
import torch.nn.functional as F  # Parameterless functions, like (some) activation functions
from torch.utils.data import (
    DataLoader,
)  # Gives easier dataset managment by creating mini batches etc.
import torchvision.datasets as datasets  # Standard datasets
import torchvision.transforms as transforms  # Transformations we can perform on our dataset for augmentation

Create a Fully-Connected NN:

Assume the input dimension is $n$, the output dimension is $m$, batch size is $b$, then $X$ has dimension $b \times n$, $A$ has dimension $m \times n$, and the result $Y$ has dimension $b \times m$.

`nn.Linear()` applies a linear transformation to the incoming data: $ Y = X A^T $

In [17]:
class NN(nn.Module):
    def __init__(self, input_size, num_classes):
        # calls the initialization function of parent nn.Module
        super(NN, self).__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, num_classes)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

Set device:

In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Define hyperparameters:

In [19]:
input_size = 784
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 3  # increase to boost performance

Load Data:

In [20]:
train_dataset = datasets.MNIST(
    root="dataset/", train=True, transform=transforms.ToTensor(), download=True
)
test_dataset = datasets.MNIST(
    root="dataset/", train=False, transform=transforms.ToTensor(), download=True
)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

Initialize our NN:

In [21]:
model = NN(input_size=input_size, num_classes=num_classes).to(device)

Loss function and optimization function:

In [22]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

Train the NN:

In [23]:
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(train_loader):
        # get data to CUDA if possible
        data = data.to(device=device)
        targets = targets.to(device=device)

        # print(data.shape) --> torch.Size([64, 1, 28, 28])
        # convert to torch.Size([64, 28^2])  flatten
        data = data.reshape(data.shape[0], -1)

        # forward pass
        pred = model(data)

        # evaluate the loss
        loss = criterion(pred, targets)

        # backward
        # Gradients by default add up; to prevent double-counting, we explicitly zero them at each iteration.
        optimizer.zero_grad()  # set all gradient to zero for each batch
        # Backpropagate the prediction loss with a call to loss.backward().
        # PyTorch deposits the gradients of the loss w.r.t. each parameter.
        loss.backward()  # compute the gradient

        # gradient descent or backward step
        # Performs a single optimization step (parameter update).
        optimizer.step()

Check accuracy on training and testing dataset:

In [24]:
def check_accuracy(loader, model):
    if loader.dataset.train:
        print("checking accuracy on training data")
    else:
        print("checking accuracy on testing data")

    num_correct = 0
    num_samples = 0
    # TODO
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            x = x.reshape(x.shape[0], -1)

            res = model(x)
            # the predictions are the indices
            _, pred = res.max(1)
            num_correct += (y == pred).sum()
            num_samples += pred.shape[0]
        print(f"Got {num_correct} / {num_samples} with accuracy {float(num_correct)/ num_samples * 100:.2f}")

In [25]:
check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

checking accuracy on training data
Got 59093 / 60000 with accuracy 98.49
checking accuracy on testing data
Got 9711 / 10000 with accuracy 97.11
